In [1]:
import pandas as pd
import requests
import matplotlib.pyplot as plt
import numpy as np

In [2]:
df_amazon = pd.read_csv('dataset/textual/raw/amazon_products.csv')
df_categories = pd.read_csv('dataset/textual/raw/amazon_categories.csv')

In [3]:
PRODUCTS_PER_CATEGORY = 416

In [4]:
df_amazon.head(2)

,asin,title,imgUrl,productURL,stars,reviews,price,listPrice,category_id,isBestSeller,boughtInLastMonth
0,B014TMV5YE,"Sion Softside Expandable Roller Luggage, Black...",https://m.media-amazon.com/images/I/815dLQKYIY...,https://www.amazon.com/dp/B014TMV5YE,4.5,0,139.99,0.00,104,False,2000
1,B07GDLCQXV,Luggage Sets Expandable PC+ABS Durable Suitcas...,https://m.media-amazon.com/images/I/81bQlm7vf6...,https://www.amazon.com/dp/B07GDLCQXV,4.5,0,169.99,209.99,104,False,1000


In [5]:
df_amazon.drop(columns=['listPrice'], inplace=True)
df_amazon.head(2)

,asin,title,imgUrl,productURL,stars,reviews,price,category_id,isBestSeller,boughtInLastMonth
0,B014TMV5YE,"Sion Softside Expandable Roller Luggage, Black...",https://m.media-amazon.com/images/I/815dLQKYIY...,https://www.amazon.com/dp/B014TMV5YE,4.5,0,139.99,104,False,2000
1,B07GDLCQXV,Luggage Sets Expandable PC+ABS Durable Suitcas...,https://m.media-amazon.com/images/I/81bQlm7vf6...,https://www.amazon.com/dp/B07GDLCQXV,4.5,0,169.99,104,False,1000


In [6]:
category_counts = df_amazon['category_id'].value_counts()
category_counts

category_id
91     28619
84     24660
270    20846
114    19822
118    18994
       ...  
195       76
186       50
185       42
102       40
194       22
Name: count, Length: 248, dtype: int64

In [ ]:
# SELECT CATEGORIES WITH MORE THAN (THE DEFINED NUMBER OF) PRODUCTS
valid_categories = category_counts[category_counts > PRODUCTS_PER_CATEGORY].index

# CREATE A LIST OF CATEGORIES. PICK 20 FROM THEM AND SAVE TO CSV - MANUALLY.
# choose_cateogies = pd.DataFrame(valid_categories, columns=['category_id']).merge(df_categories, left_on='category_id', right_on='id', how='left').reset_index(drop=True).drop(columns=['id'])
# choose_cateogies.to_csv('dataset/textual/temp/choose_categories.csv', index=False)

valid_categories = pd.read_csv('dataset/textual/filtered/chosen_categories.csv')
valid_categories.head(2)

,category_id,category_name
0,116,Women's Clothing
1,114,Men's Shoes


In [8]:
# SELECT PRODUCTS FROM THE SELECTED CATEGORIES
selected_products = df_amazon[df_amazon['category_id'].isin(valid_categories['category_id'])].groupby('category_id').apply(lambda x: x.sample(n=PRODUCTS_PER_CATEGORY)).reset_index(drop=True)
selected_products.head(2)

C:\Users\ShahrukhAzharAhsan\AppData\Local\Temp\ipykernel_69056\1394958067.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  selected_products = df_amazon[df_amazon['category_id'].isin(valid_categories['category_id'])].groupby('category_id').apply(lambda x: x.sample(n=PRODUCTS_PER_CATEGORY)).reset_index(drop=True)


,asin,title,imgUrl,productURL,stars,reviews,price,category_id,isBestSeller,boughtInLastMonth
0,B0BYZQYC3H,New Slim 230W AC Adapter Replacement for Lenov...,https://m.media-amazon.com/images/I/71EfaT-wM1...,https://www.amazon.com/dp/B0BYZQYC3H,3.5,0,39.99,65,False,0
1,B0C84T7B22,100Pcs Forest Stickers | Forest Trees Animals ...,https://m.media-amazon.com/images/I/81CmApDXnO...,https://www.amazon.com/dp/B0C84T7B22,0.0,0,6.99,65,False,50


In [9]:
selected_products.shape

(4992, 10)

In [10]:
# CHECK THE NUMBER OF CATEGORIES
selected_products.groupby('category_id').size().shape

(12,)

In [11]:
# GET THE CATEGORY NAMES FOR THE SELECTED PRODUCTS
selected_products = selected_products.merge(df_categories, left_on='category_id', right_on='id',how='left').reset_index(drop=True)
selected_products.head(2)

,asin,title,imgUrl,productURL,stars,reviews,price,category_id,isBestSeller,boughtInLastMonth,id,category_name
0,B0BYZQYC3H,New Slim 230W AC Adapter Replacement for Lenov...,https://m.media-amazon.com/images/I/71EfaT-wM1...,https://www.amazon.com/dp/B0BYZQYC3H,3.5,0,39.99,65,False,0,65,Laptop Accessories
1,B0C84T7B22,100Pcs Forest Stickers | Forest Trees Animals ...,https://m.media-amazon.com/images/I/81CmApDXnO...,https://www.amazon.com/dp/B0C84T7B22,0.0,0,6.99,65,False,50,65,Laptop Accessories


In [12]:
# CHECK IF THERE ARE EQUAL NUMBER OF PRODUCTS IN EACH CATEGORY
selected_products.groupby('category_name').size()

category_name
Backpacks                       416
Dolls & Accessories             416
Girls' Shoes                    416
Headphones & Earbuds            416
Home Storage & Organization     416
Laptop Accessories              416
Men's Shoes                     416
Sports & Fitness                416
Televisions & Video Products    416
Toys & Games                    416
Women's Clothing                416
Women's Handbags                416
dtype: int64

In [13]:
# Save the final selected products\
selected_products.to_csv('dataset/textual/filtered/selected_products.csv', index=False)

Images

In [14]:
# SAVE THE IMAGES OF THE SELECTED PRODUCTS (AND RENAME THEM WITH THE ASIN)
for row in selected_products[['imgUrl', 'asin']].head(100).itertuples():
    image_link = row.imgUrl
    # print(image_link)
    renamed_image = row.asin + ".jpg"
    print(renamed_image)
    img_data = requests.get(image_link).content
    with open('dataset/images/' + renamed_image, 'wb') as handler:
        handler.write(img_data)

B0BYZQYC3H.jpg
B0C84T7B22.jpg
B0C587GT98.jpg
B01DNF56NO.jpg
B08KWKRGPC.jpg
B07D583NGN.jpg
B07H5JHWN9.jpg
B01FQAEVLM.jpg
B07KSTMM44.jpg
B0CD4FHNRN.jpg
B07BSB9F2C.jpg
B08B7RWVT5.jpg
B085T959R7.jpg
B07X5H9NMN.jpg
B09XHRCMPH.jpg
B09T622PGW.jpg
B0BJDQT9ZV.jpg
B0784FK71L.jpg
B0871W9RZQ.jpg
B08MK9T6BH.jpg
B09SH68735.jpg
B07SSRT9RZ.jpg
B08LG2S5XY.jpg
B07VD3B14F.jpg
B08LQ4NCLZ.jpg
B075NVG8X6.jpg
B09ZY4LRDJ.jpg
B0BN12W7KB.jpg
B0BLV5HVSH.jpg
B0BV6JG7SW.jpg
B09728137S.jpg
B0C37WP2PT.jpg
B0C2Y24N6W.jpg
B09LHKM4SY.jpg
B0BZY27RD3.jpg
B0C6LMP6BY.jpg
B0BPB8V84T.jpg
B0BKGVTT21.jpg
B0BW8S98QP.jpg
B09Y157KTV.jpg
B0BRHXZDWG.jpg
B089ZXL96Y.jpg
B0BB9CXN7J.jpg
B0BVQQMNXS.jpg
B08B1J3LBF.jpg
B0B9RJ6YBM.jpg
B0C3XH5MR3.jpg
B01MXW9OMN.jpg
B0BPSMM86F.jpg
B0BJ23PKCP.jpg
B09M7ZZLXW.jpg
B07D848J6V.jpg
B0BG2V5YDF.jpg
B07Y8C3KDT.jpg
B0BX9MMP8L.jpg
B0BMLDST96.jpg
B0C7KCN7XB.jpg
B08YJSFN4T.jpg
B09VT87RBH.jpg
B0CC29M6H9.jpg
B0C9Q788YC.jpg
B0B2QS6LTY.jpg
B0C23N3556.jpg
B0C7BXNYZL.jpg
B0BJV4S4C2.jpg
B09V9TDL9M.jpg
B0CDW79TNL